# 1) Descentes de gradient sans contraintes
-------------------------------------------


1-1) Soit la famille de fonction: $f(x, y) = \frac{x^2}{2} + \frac{ay^2}{2} ~\forall~ a \in [1; +\inf]$

Le nombre de conditionnement d'une fonction de cette famille est équivalent au nombre de conditionnement de sa Hessienne:

$H = \begin{pmatrix} 1 & 0 \\ 0 & a \end{pmatrix}$

On choisit d'utiliser la norme $l_2$ pour calculer le nombre de conditionnement de notre matrice.
Dans ce cadre, le nombre de conditionnement est égal à: $||H||_2 \times ||H^{-1}||_2$

Dans notre cas, le nombre de conditionnement correspond à la plus grande valeur singulière divisée par la plus petite. Donc le nombre de conditionnement est $\frac{a}{1} = a$ avec $a \in [1; +\inf[$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
from scipy import misc

In [ ]:
def partial_derivative(ob_function, point, var=0):
    args = point
    def wraps(x):
        args[var] = x
        return ob_function(args)
    return misc.derivative(wraps, point[var], dx=1e-6)

In [ ]:
def d_gd(ob_function, x):
    dim = x.shape[0]
    gradient = partial_derivative(ob_function, x, 0)
    for i in range(1, dim):
        gradient = np.vstack((gradient, partial_derivative(ob_function, x, i)))
    return gradient

def d_sgd(ob_function, x):
    dim = x.shape[0]
    sdescent = np.array([0]*dim, dtype=float).reshape(-1, 1)
    max_diff, imax_diff = sdescent[0], 0
    for i in range(dim):
        diff_i = np.linalg.norm(partial_derivative(ob_function, x, i), 2)
        if  diff_i > max_diff:
            imax_diff, max_diff = i, diff_i
    sdescent[imax_diff] = max_diff
    return sdescent

In [ ]:
def build_obj(a):
    def obj_func(x):
        return (x[0] ** 2)/2 + a*(x[1] ** 2)/2
    return obj_func

def gradient_descent(x, ob_function, d_direction, 
                     rate=(lambda x, y: 0.01),
                     decay_function=(lambda x, y: abs(x[0] - y[0])),
                     tolerance=0.0001, max_iter=1000,
                     plot_f=(lambda x, y: None),
                     verbose=False,
                     norm=2):
    """Gradient Descent.
    
    Computes minimal value of a convex function and local minimum of none convex function.
    
    Args:
        x (ndarray): initial starting point for descent.
        ob_function: objective function of optimisation problem, has input ndarray and outputs float.
        d_direction: function computing descent direction, outputs ndarray.
        rate: function computing learning rate, outputs float.
        decay_function: function computing decay, outputs float.
        tolerance (float): slack tolerance.
        max_iter (int): upper bound on number of iterations.      
        plot_f: plotting function for iteration points.
         
    Output:
        (int, int) minimizer, minimal value.
        
    """
    n_iter = 0
    decay = tolerance + 1
    y = ob_function(x)
    plot_f(x, y)
    while decay > tolerance and n_iter < max_iter:
        grad = d_direction(ob_function, x)
        x = x - rate(ob_function, n_iter) * grad
        decay = np.linalg.norm(grad, norm)
        n_iter += 1
        y = ob_function(x)
        plot_f(x, y)
    if verbose:
        print(' Iteration nu. = {}\n approx. = {}\n ob value = {}\n and decay = {}.'.format(n_iter, x.flatten(), y[0], decay))
    if decay > tolerance:
        warnings.warn("Decay didn't get under tolerance rate.", RuntimeWarning)
    return ((x, y), n_iter)

In [ ]:
def eval_nb_iterations(step, k=0, set_size=60, max_value=50, start=np.array([[1], [1]], dtype=float)):

    a_values = np.linspace(1, max_value, num=set_size)
    iterations = np.zeros((set_size, 1))
    conditioning = np.zeros((set_size, 1))
    results = np.zeros((set_size, 1))

    count = 0
    for a in a_values:
        m = np.array([[1, 0], [0, a]])
        nb_cond1 = np.linalg.norm(m, 2) * np.linalg.norm(np.linalg.inv(m), 2)
        conditioning[count, 0] = nb_cond1
    
        obj = build_obj(a)

        (x, y), n_iter = gradient_descent(start, obj, d_sgd, 
                                     rate=(lambda x, y: step),
                                     tolerance=0.0001, max_iter=1000,
                                     plot_f=(lambda x, y: None))
        iterations[count, 0] = n_iter
        results[count, 0] = y[0]
        count+=1

    plt.plot(results, 'g', iterations, 'r', conditioning, 'b')
    plt.show()

**Pour un pas de 0.1**

In [ ]:
eval_nb_iterations(0.1, set_size=50, max_value=20)

On remarque deux choses pour notre famille:
- Le nombre d'itérations de la descente de gradient diminue quand le nombre de conditionnement augmente.
- A partir d'un certain nombre de conditionnement limite, la descente de gradient diverge.

Voici un exemple de divergence de l'algorithme pour les nombres de conditionnement trop élevés. Les valeurs objectives, tracées en vert, sont très éloignée de 0.

In [ ]:
eval_nb_iterations(0.1, set_size=50, max_value=30)

On effectue maintenant des tests pour plusieurs pas différents:

**Pour un pas de 0.15:**

In [ ]:
eval_nb_iterations(0.15, set_size=50, max_value=10)

**Pour un pas de 0.05:**

In [ ]:
eval_nb_iterations(0.05, set_size=50, max_value=10)

**Pour un pas de 0.035:**

In [ ]:
eval_nb_iterations(0.035, set_size=50, max_value=10)

In [ ]:
a = 3
m = np.array([[1, 0], [0, a]])
nb_cond1 = np.linalg.norm(m, 2) * np.linalg.norm(np.linalg.inv(m), 2)

obj = build_obj(a)

start = np.array([[1], [1]], dtype=float)
P, n_iter = gradient_descent(start, obj, d_sgd, 
                             rate=(lambda x, y: 0.15),
                             decay_function=(lambda x, y: abs(x[0] - y[0])),
                             tolerance=0.0001, max_iter=1000,
                             plot_f=(lambda x, y: None),
                             verbose=True)

a = 8
m = np.array([[1, 0], [0, a]])
nb_cond1 = np.linalg.norm(m, 2) * np.linalg.norm(np.linalg.inv(m), 2)

obj = build_obj(a)

start = np.array([[1], [1]], dtype=float)
P, n_iter = gradient_descent(start, obj, d_sgd, 
                             rate=(lambda x, y: 0.15),
                             decay_function=(lambda x, y: abs(x[0] - y[0])),
                             tolerance=0.0001, max_iter=1000,
                             plot_f=(lambda x, y: None),
                             verbose=True)


On remarque que la précision des valeurs trouvées est sensiblement la même pour des nombres de conditionnement identiques. Cependant, choisir un nombre de conditionnement plus grand permet de choisir un step plus faible sans impacter trop fortement les performances. De cette manière, il est peut-être possible d'augmenter la précision de la valeur objective finale.

**1-2) Comparaison nomres L1 - L2 **

In [ ]:
def eval_nb_iterations_l1_l2(step, set_size=60, max_value=50):

    a_values = np.linspace(1, max_value, num=set_size)
    iterations = np.zeros((set_size, 1))
    iterations2 = np.zeros((set_size, 1))
    conditioning = np.zeros((set_size, 1))

    count = 0
    for a in a_values:
        m = np.array([[1, 0], [0, a]])
        nb_cond1 = np.linalg.norm(m, 2) * np.linalg.norm(np.linalg.inv(m), 2)
        conditioning[count, 0] = nb_cond1
    
        obj = build_obj(a)
        derivative_obj = build_derivative_obj(a)

        start = np.array([[1], [1]], dtype=float)
        P, n_iter = gradient_descent(start, obj, d_sgd, 
                                     rate=(lambda x, y: step),
                                     decay_function=(lambda x, y: abs(x[0] - y[0])),
                                     tolerance=0.0001, max_iter=1000,
                                     plot_f=(lambda x, y: None))
        P2, n_iter2 = gradient_descent(start, obj, d_sgd, 
                                     rate=(lambda x, y: step),
                                     decay_function=(lambda x, y: np.sqrt(x[0]**2 - y[0]**2)),
                                     tolerance=0.0001, max_iter=1000,
                                     plot_f=(lambda x, y: None))
        iterations[count, 0] = n_iter
        iterations2[count, 0] = n_iter2
        count+=1

    # red = L1-norm
    # green = L2-norm
    plt.plot(iterations, 'r', conditioning, 'b', iterations2, 'g')
    plt.show()

In [ ]:
eval_nb_iterations_l1_l2(0.025, 50, 40)

In [ ]:
eval_nb_iterations_l1_l2(0.05, 20, 30)

**1-3) learning rate scheduling**

Le but ici est de proposer une méthode pour faire évoluer le rate. Dans notre cas, nous allons implémenter un rate décroissant en fonction du nombre d'itérations. Cette méthode devrait permettre de choisir un taux plus fort et donc de converger plus vite. Elle introduit cependant le risque de réduire le rate trop rapidement.

$r_0$ et $k$ sont des hyperparamètres pour cet algorithme.

In [ ]:
def plot_2d(ob_function, xmin=-4, xmax=4, ymin=-2, ymax=2, nb_pts_h=50, nb_pts_v=50):
    x1 = np.linspace(xmin, xmax, nb_pts_h)
    x2 = np.linspace(ymin, ymax, nb_pts_v)
    X, Y = np.meshgrid(x1, x2)
    Z = ob_function(np.stack((X,Y), axis=0))
    plt.contour(X, Y, Z, cmap='RdGy')

plt.figure(figsize=(10,10))
plot_2d(obj)

step = 0.3
k = 0.015

start=np.array([[-2], [1.5]], dtype=float)
a = 8

m = np.array([[1, 0], [0, a]])
nb_cond1 = np.linalg.norm(m, 2) * np.linalg.norm(np.linalg.inv(m), 2)

obj = build_obj(a)
derivative_obj = build_derivative_obj(a)

(x, y), n_iter = gradient_descent(start, obj, d_gd, 
                                  rate=(lambda x, y: step * np.exp(-k * y)),
                                  tolerance=0.0001, max_iter=1000,
                                  plot_f=(lambda x, y:plt.scatter(x[0], x[1], c='k')))
result = y

print("result: " + str(result))
print("start step: " + str(step))
print("iterations: " + str(n_iter))

In [ ]:
eval_nb_iterations(0.3, set_size=50, max_value=10)

On peut remarquer que pour un rate de départ de 0.3, avec des nombres de conditionnement supérieurs à 7, la descente sans learning rate scheduling diverge. La méthode avec learning rate schedulin permet de corriger cet état de fait.

**1-4) descentes de gradient avancées**

Descente accélérée de Nesterov:

In [ ]:
def nesterov_gradient_descent(x, ob_function, d_direction, 
                     rate=(lambda x, y: 0.01),
                     decay_function=(lambda x, y: abs(x[0] - y[0])),
                     tolerance=0.0001, max_iter=1000,
                     plot_f=(lambda x, y: None),
                     verbose=False):
    n_iter = 0
    decay = tolerance + 10  # Make sure that we get into first loop
    
    y = ob_function(x)
    y_t = x
    t = 1
    
    while decay > tolerance and n_iter < max_iter:
        curr_x = x - rate(ob_function, n_iter) * d_direction(ob_function, y_t)
        curr_t = (1 + np.sqrt(1 + 4 * t**2)) / 2
        curr_y = curr_x + ((t - 1)/curr_t) * (x - curr_x)
        
        y_t = curr_y
        x = curr_x
        t = curr_t
        
        tmp_y = ob_function(x)
        decay = decay_function(y, tmp_y)
        y = tmp_y
        n_iter += 1
        plot_f(x, y)
    if verbose:
        print(' Iteration nu. = {}\n approx. = {}\n ob value = {}\n and decay = {}.'.format(n_iter, x.flatten(), y[0], decay))
    return ((x, y), n_iter) if decay <= tolerance else warnings.warn("Decay didn't get under tolerance rate.", RuntimeWarning)

In [ ]:
def eval_nb_iterations_nesterov(step, set_size=60, max_value=50):

    a_values = np.linspace(1, max_value, num=set_size)
    iterations = np.zeros((set_size, 1))
    iterations2 = np.zeros((set_size, 1))
    conditioning = np.zeros((set_size, 1))

    count = 0
    for a in a_values:
        m = np.array([[1, 0], [0, a]])
        nb_cond1 = np.linalg.norm(m, 2) * np.linalg.norm(np.linalg.inv(m), 2)
        conditioning[count, 0] = nb_cond1
    
        obj = build_obj(a)
        derivative_obj = build_derivative_obj(a)

        start = np.array([[1], [1]], dtype=float)
        P, n_iter = nesterov_gradient_descent(start, obj, d_sgd, 
                                     rate=(lambda x, y: step),
                                     decay_function=(lambda x, y: abs(x[0] - y[0])),
                                     tolerance=0.0001, max_iter=1000,
                                     plot_f=(lambda x, y: None))
        P2, n_iter2 = gradient_descent(start, obj, d_sgd, 
                                     rate=(lambda x, y: step),
                                     decay_function=(lambda x, y: abs(x[0] - y[0])),
                                     tolerance=0.0001, max_iter=1000,
                                     plot_f=(lambda x, y: None))
        iterations[count, 0] = n_iter
        iterations2[count, 0] = n_iter2
        count+=1

    # red = nesterov
    # green = classic
    plt.plot(iterations, 'r', conditioning, 'b', iterations2, 'g')
    plt.show()

In [ ]:
eval_nb_iterations_nesterov(0.01, 10, 10)

In [ ]:
eval_nb_iterations_nesterov(0.01, 20, 100)

In [ ]:
eval_nb_iterations_nesterov(0.01, 50, 1000)